In [1]:
pip install spacy

   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.1 MB 660.6 kB/s eta 0:00:19
   ---------------------------------------- 0.1/12.1 MB 544.7 kB/s eta 0:00:23
   ---------------------------------------- 0.1/12.1 MB 573.4 kB/s eta 0:00:21
   ---------------------------------------- 0.1/12.1 MB 595.3 kB/s eta 0:00:21
   ---------------------------------------- 0.1/12.1 MB 607.9 kB/s eta 0:00:20
    --------------------------------------- 0.2/12.1 MB 583.1 kB/s eta 0:00:21
    --------------------------------------- 0.2/12.1 MB 621.6 kB/s eta 0:00:20
    --------------------------------------- 0.2/12.1 MB 654.2 kB/s eta 0:00:19
    --------------------------------------- 0.2/12.1 MB 654.2 kB/s eta 0:00:19
    --------------------------------------- 0.2/12.1 MB 654.2 kB/s eta 0:00:19
    --------------------------------------- 0.2/12.1 MB 654.2 kB/s eta 0:00:19
    --------------------------------------- 0.3/12.1 MB 520.7

In [2]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 640.0 kB/s eta 0:00:20
     --------------------------------------- 0.0/12.8 MB 487.6 kB/s eta 0:00:27
     --------------------------------------- 0.1/12.8 MB 544.7 kB/s eta 0:00:24
     --------------------------------------- 0.1/12.8 MB 544.7 kB/s eta 0:00:24
     --------------------------------------- 0.1/12.8 MB 544.7 kB/s eta 0:00:24
      -------------------------------------- 0.2/12.8 MB 769.9 kB/s eta 0:00:17
      -------------------------------------- 0.3/12.8 MB 930.9 kB/s eta 0:00:14
      -------------------------------------- 0.3/12.8 MB 930.9 kB/s eta 0:00:14
      -------------------------------------- 0.3/12.8 MB 930.9 kB/s eta 0:00:14
      -------------------------------------- 0.3/12.8 MB 930.9 kB/s eta 0:00:14
      -------------------------------------- 0.3/12.8 MB 930.9 kB/s eta 0:00:14
      -------------------------------------- 0.3

## to split given txt into columns and add a new custom weapon column too

### if u want to keep new column like weapons say like uniq text pattern etc then use separate matchers as written below

In [4]:
import spacy
import pandas as pd
from spacy.matcher import Matcher

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize separate matchers for weapons and unique text
weapon_matcher = Matcher(nlp.vocab)
unique_text_matcher = Matcher(nlp.vocab)

# Define custom patterns for weapons
weapon_patterns = [
    [{"LOWER": {"IN": ["pistol", "revolver", "rifle", "bulldozer", "shotgun", "uzi", "mp5", "knife", "katana", "sabre", "axe", "tomahawk", "dagger", "stiletto", "dirk", "grenade", "bomb", "club", "bat", "spear", "halberd", "bow", "crossbow", "taser", "stun", "gun", "pepper", "spray", "rubber", "bullet"]}}],
    [{"LOWER": "assault"}, {"LOWER": "rifle"}],
    [{"LOWER": "sniper"}, {"LOWER": "rifle"}],
    [{"LOWER": "submachine"}, {"LOWER": "gun"}],
    [{"LOWER": "machine"}, {"LOWER": "gun"}],
    [{"LOWER": "combat"}, {"LOWER": "knife"}],
    [{"LOWER": "bowie"}, {"LOWER": "knife"}],
    [{"LOWER": "landmine"}],
    [{"LOWER": "naval"}, {"LOWER": "mine"}],
    [{"LOWER": "improvised"}, {"LOWER": "explosive"}, {"LOWER": "device"}],
    [{"LOWER": "howitzer"}],
    [{"LOWER": "mortar"}],
    [{"LOWER": "main"}, {"LOWER": "battle"}, {"LOWER": "tank"}],
]

# Add patterns for weapons to the weapon matcher
weapon_matcher.add("WEAPON", weapon_patterns)

# Define custom patterns for another column (e.g., UniqueTextColumn)
unique_text_patterns = [
   [{"LOWER": {"IN": ["is","it"]}}
]]

# Add patterns for UniqueTextColumn to the unique text matcher
unique_text_matcher.add("UNIQUE_TEXT", unique_text_patterns)

# Function to extract segments from each line
def extract_segments(line):
    doc = nlp(line)  # Process line using spaCy
    entities = {ent.label_: ent.text for ent in doc.ents}  # Extract named entities
    
    # Initialize sets to store unique weapons and unique texts detected in the line
    unique_weapons = set()
    unique_texts = set()

    # Apply the weapon matcher to the doc
    weapon_matches = weapon_matcher(doc)
    for match_id, start, end in weapon_matches:
        span = doc[start:end]
        unique_weapons.add(span.text)

    # Apply the unique text matcher to the doc
    unique_text_matches = unique_text_matcher(doc)
    for match_id, start, end in unique_text_matches:
        span = doc[start:end]
        unique_texts.add(span.text)
    
    # Add the list of unique weapons and unique texts to the entities dictionary
    entities["WEAPON"] = list(unique_weapons)
    entities["UNIQUE_TEXT"] = list(unique_texts)
    entities["Spoken"] = line.strip()
    
    return entities  # Return dictionary with entities

# Read your text file
with open(r"C:\Users\HEEMA SAMEERA\OneDrive\Desktop\SampleArticles[1].txt", encoding='utf-8') as file:
    lines = file.readlines()

# Initialize a list to store extracted segments
data = []

# Iterate over each line and extract segments
for line in lines:
    data.append(extract_segments(line))

# Create a DataFrame
df = pd.DataFrame(data)

# Reorder columns to have 'Spoken' column first
spoken_column = df.pop('Spoken')
df = pd.concat([spoken_column, df], axis=1)

# Convert the lists of weapons and unique texts to comma-separated strings
df['WEAPON'] = df['WEAPON'].apply(lambda x: ', '.join(x))
df['UNIQUE_TEXT'] = df['UNIQUE_TEXT'].apply(lambda x: ', '.join(x))

# Display the DataFrame
print(df)

# # Save DataFrame to an Excel file
# file_path = 'output.xlsx'
# df.to_excel(file_path, index=False)


                                               Spoken            GPE  \
0   New Delhi, May 17Prime Minister Narendra Modi ...       Pakistan   
1                                                                NaN   
2   Jupinderjit Singh amp; Gurbaxpuri Khadoor Sahi...          India   
3   External affairs minister S Jaishankar, amidst...         Kuwait   
4                                                                NaN   
5                                                                NaN   
6   To enjoy additional benefitsCONNECT WITH US Pu...          India   
7   Asia is witnessing new tensions in land and se...         Russia   
8   Azamgarh/Bhadohi: Prime Minister Narendra Modi...          Delhi   
9   Asia is witnessing new tensions in land and se...         Russia   
10  Amid escalating tensions in Asia, External Aff...          India   
11  To enjoy additional benefitsCONNECT WITH USUpd...         Russia   
12  India now takes not just clear, but confident ...           

In [ ]:
# import pandas as pd
# import spacy
# from spacy.matcher import Matcher

# # Load spaCy model
# nlp = spacy.load("en_core_web_sm")

# # Initialize matcher for weapons
# weapon_matcher = Matcher(nlp.vocab)

# # Define custom patterns for weapons
# weapon_patterns = [
#     # Define your custom weapon patterns here...
# ]

# # Add patterns for weapons to the weapon matcher
# weapon_matcher.add("WEAPON", weapon_patterns)

# # Initialize matcher for pronouns
# pronoun_matcher = Matcher(nlp.vocab)

# # Define custom patterns for pronouns
# pronoun_patterns = [
#     # Define your custom pronoun patterns here...
# ]

# # Add patterns for pronouns to the pronoun matcher
# pronoun_matcher.add("PRONOUN", pronoun_patterns)

# # Function to extract weapons from each line
# def extract_weapons(line):
#     doc = nlp(line)  # Process line using spaCy
#     unique_weapons = set()

#     # Apply the weapon matcher to the doc
#     weapon_matches = weapon_matcher(doc)
#     for match_id, start, end in weapon_matches:
#         span = doc[start:end]
#         unique_weapons.add(span.text)

#     return ', '.join(unique_weapons) if unique_weapons else None  # Return unique weapons as comma-separated string or None if no weapons detected

# # Function to extract pronouns from each line
# def extract_pronouns(line):
#     doc = nlp(line)  # Process line using spaCy
#     unique_pronouns = set()

#     # Apply the pronoun matcher to the doc
#     pronoun_matches = pronoun_matcher(doc)
#     for match_id, start, end in pronoun_matches:
#         span = doc[start:end]
#         unique_pronouns.add(span.text)

#     return ', '.join(unique_pronouns) if unique_pronouns else None  # Return unique pronouns as comma-separated string or None if no pronouns detected

# # Read your CSV file
# df = pd.read_csv("your_file.csv")

# # Initialize lists to store extracted weapons and pronouns
# weapons = []
# pronouns = []

# # Iterate over each row and extract weapons and pronouns from each column
# for index, row in df.iterrows():
#     weapons_in_row = []
#     pronouns_in_row = []
#     for column in df.columns:
#         line = str(row[column])  # Convert cell value to string
#         weapons_in_cell = extract_weapons(line)
#         pronouns_in_cell = extract_pronouns(line)
#         if weapons_in_cell:
#             weapons_in_row.extend(weapons_in_cell)
#         if pronouns_in_cell:
#             pronouns_in_row.extend(pronouns_in_cell)
#     weapons.append(', '.join(weapons_in_row) if weapons_in_row else None)  # Combine weapons from all columns
#     pronouns.append(', '.join(pronouns_in_row) if pronouns_in_row else None)  # Combine pronouns from all columns

# # Add the consolidated weapons and pronouns as new columns to the DataFrame
# df['Consolidated_WEAPONS'] = weapons
# df['Unique_Pronouns'] = pronouns

# # Display the DataFrame
# print(df)

# # # Save DataFrame to an Excel file
# # df.to_excel('output.xlsx', index=False)
